In [3]:
from pymemcache.client.hash import HashClient

# Criar um cluster distribuído com 3 nós Memcached
cluster1 = HashClient([
    ('localhost', 11211),
    ('localhost', 11212),
    ('localhost', 11213),
])

# Armazenar e recuperar um valor distribuído no cluster
cluster1.set('chave_distribuida', 'Dado em cluster!')
print(cluster1.get('chave_distribuida').decode())

Dado em cluster!


In [ ]:
import json
from pymemcache.client.hash import HashClient

# Criar um cluster distribuído com 3 nós Memcached
servers = [
    ('localhost', 11211),
    ('localhost', 11212),
    ('localhost', 11213),
]
cluster = HashClient(servers)

# Função de debug para saber qual servidor armazena uma chave (método interno)
def get_server_for_key(client, key):
    server = client._get_server(key)  # Atenção: método interno, para debug apenas.
    return server

# Simular dados de produtos (exemplo de e-commerce)
products = {
    "prod_1": {"id": 1, "name": "Laptop", "price": 2500},
    "prod_2": {"id": 2, "name": "Smartphone", "price": 1200},
    "prod_3": {"id": 3, "name": "Headphones", "price": 300},
    "prod_4": {"id": 4, "name": "Smartwatch", "price": 500},
    "prod_5": {"id": 5, "name": "Headphones da Loja", "price": 600},
    "prod_6": {"id": 6, "name": "Smartwatch da Loja", "price": 10000000},
}

print("=== Armazenando produtos no cluster ===")
# Armazenar cada produto como JSON com expiração de 1 hora (3600 segundos)
for key, product in products.items():
    data = json.dumps(product)
    cluster.set(key, data, expire=120)
    print(f"Produto {key} armazenado.")

print("\n=== Recuperando os produtos armazenados ===")
# Recuperar e exibir os produtos
for key in products.keys():
    value = cluster.get(key)
    if value:
        product_data = json.loads(value.decode())
        print(f"{key}: {product_data}")
    else:
        print(f"{key}: Não encontrado no cache.")

# Testar o acesso a uma chave que não existe
non_existent = cluster.get("non_existent_key")
if non_existent is None:
    print("\nA chave 'non_existent_key' não existe no cluster.")

print("\n=== Atualizando e Deletando produtos ===")
# Atualizar o produto 2
updated_product = {"id": 2, "name": "Smartphone Pro", "price": 1500}
cluster.set("prod_2", json.dumps(updated_product), expire=3600)
print("Produto prod_2 atualizado.")

# Deletar o produto 3
cluster.delete("prod_3")
print("Produto prod_3 deletado.")

# Verificar a deleção
if cluster.get("prod_3") is None:
    print("Produto prod_3 foi removido com sucesso.")

print("\n=== Operações de Incremento (contador de views) ===")
# Configurar um contador para visualizações de um produto, pode ser necessário quantas vezes o produto está a ser acessado para monitorar e analisar o tráfego ou populariadade de conteúdos
cluster.set("views_prod_1", "0")
cluster.incr("views_prod_1", 1)
cluster.incr("views_prod_1", 1)
views = cluster.get("views_prod_1")
print("Views para prod_1:", views.decode())


=== Armazenando produtos no cluster ===
Produto prod_1 armazenado.
Produto prod_2 armazenado.
Produto prod_3 armazenado.
Produto prod_4 armazenado.
Produto prod_5 armazenado.
Produto prod_6 armazenado.

=== Recuperando os produtos armazenados ===
prod_1: {'id': 1, 'name': 'Laptop', 'price': 2500}
prod_2: {'id': 2, 'name': 'Smartphone', 'price': 1200}
prod_3: {'id': 3, 'name': 'Headphones', 'price': 300}
prod_4: {'id': 4, 'name': 'Smartwatch', 'price': 500}
prod_5: {'id': 5, 'name': 'Headphones da Loja', 'price': 600}
prod_6: {'id': 6, 'name': 'Smartwatch da Loja', 'price': 10000000}

A chave 'non_existent_key' não existe no cluster.

=== Atualizando e Deletando produtos ===
Produto prod_2 atualizado.
Produto prod_3 deletado.
Produto prod_3 foi removido com sucesso.

=== Operações de Incremento (contador de views) ===
Views para prod_1: 2
